# Controllability

This example analyzes controllability of a 2D model of a quadrotor moving in a plane, controlled by a front and rear motor.
Where $u_0$ is the front throttle and $u_1$ is the rear throttle.

$m \ddot{p_x} = (u_0 + u_1) sin(\theta)$

$m \ddot{p_z} = (u_0 + u_1) cos(\theta) - m g$

$J\ddot{\theta} = l (u_0 - u_1)$

$\vec{x} = \begin{bmatrix} p_x && p_z && \theta && \dot{p_x} && \dot{p_z} && \dot{\theta} \end{bmatrix}^T$

$\vec{u} = \begin{bmatrix} u_0 && u_1 \end{bmatrix}^T$

$\vec{y} = \vec{x}$


$\dot{\vec{x}} = f(\vec{x}, \vec{u})$

$\vec{y} = g(\vec{y}, \vec{u})$

In [11]:
import casadi as ca
import control
import numpy as np

In [12]:
u = ca.SX.sym('u', 2)

m = ca.SX.sym('m')
g = ca.SX.sym('g')
l = ca.SX.sym('l')
J = ca.SX.sym('J')
p = ca.vertcat(m, l, g, J)

p_x = ca.SX.sym('p_x')
p_z = ca.SX.sym('p_z')
v_x = ca.SX.sym('v_x')
v_z = ca.SX.sym('v_z')
omega = ca.SX.sym('omega')
theta = ca.SX.sym('theta')
x = ca.vertcat(p_x, p_z, theta, v_x, v_z, omega)

T = u[0] + u[1]
M = l*(u[0] - u[1])
rhs = ca.vertcat(v_x, v_z, omega, T*ca.sin(theta), T*ca.cos(theta) - m*g, l*M/J)
y = x


A = ca.jacobian(rhs, x)
B = ca.jacobian(rhs, u)
C = ca.jacobian(y, x)
D = ca.jacobian(y, u)

f_ss = ca.Function('ss', [x, u, p], [A, B, C, D])

In [16]:
# can solve trim easily for hover
m0 = 1
l0 = 1
J0 = 1
g0 = 9.8
theta0 = 0
T0 = m0*g0/np.cos(theta0)
M0 = 0
u0_0 = (M0/l0 + T0)/2
u1_0 = T0 - u0_0

sys_hover = control.ss(*f_ss([0, 0, 0, 0, 0, 0], [u0_0, u1_0], [m0, l0, g0, J0]))
sys_hover

A = [[ 0.   0.   0.   1.   0.   0. ]
 [ 0.   0.   0.   0.   1.   0. ]
 [ 0.   0.   0.   0.   0.   1. ]
 [ 0.   0.   9.8  0.   0.   0. ]
 [ 0.   0.  -0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]]

B = [[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 1.  1.]
 [ 1. -1.]]

C = [[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]

D = [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]

$ \dot{x} = Ax + Bu$

$R = \begin{bmatrix} B && AB && A^2B && A^3B \ldots && A^5B \end{bmatrix}$

In [20]:
np.set_printoptions(precision=3, linewidth=200)
R = control.ctrb(sys_hover.A, sys_hover.B)
R

matrix([[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  9.8, -9.8,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  1. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  9.8, -9.8,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 1. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 1. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ]])

In [21]:
np.linalg.matrix_rank(R), A.shape[0]

(6, 6)